# Import libraries

In [3]:
import pandas as pd
import numpy as np
import os

# Import Data Set

In [6]:
path = r'C:\Users\shaiv\Downloads\Instacart Basket Analysis'
path

'C:\\Users\\shaiv\\Downloads\\Instacart Basket Analysis'

### 03 Import the merged file

In [9]:
#import orders_product_combined data set
df_ords_prods_combined = pd.read_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'orders_products_combined.pkl'))

In [10]:
# import products_checked data set
df_prods = pd.read_csv(os.path.join(path, '02 Data', 'Prepared Data', 'products_checked.csv'))

### 04 Checking the shape of the imported dataframe

In [12]:
# 04 Check shape of df_ords_prods_combined
df_ords_prods_combined.shape

(32434489, 11)

In [13]:
# 04 Check shape of df_prods
df_prods.shape

(49672, 6)

##### The no of rows and column of the both data sets match the rows and column from practice exercise for orders_products_combined.pkl and previous exercise for products_checked.csv

In [15]:
# 05 check output of df_ords_prods_combined
df_ords_prods_combined.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,both


In [16]:
df_ords_prods_cleaned = df_ords_prods_combined.drop(['_merge'], axis = 1)
df_ords_prods_cleaned.shape

(32434489, 10)

In [17]:
# check output of df_prods
df_prods.head()

,Unnamed: 0,product_id,product_name,aisle_id,department_id,prices
0,0,1,Chocolate Sandwich Cookies,61,19,5.8
1,1,2,All-Seasons Salt,104,13,9.3
2,2,3,Robust Golden Unsweetened Oolong Tea,94,7,4.5
3,3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1,10.5
4,4,5,Green Chile Anytime Sauce,5,13,4.3


In [18]:
df_prods_cleaned = df_prods.drop(['Unnamed: 0'], axis = 1)
df_prods_cleaned.shape

(49672, 5)

##### orders_products_combined has the size before exporting just 1 less column because we removed the '_merge' column

### 05 Best way to combine both data sets is by merging them

##### There is a common column, 'product_id,' in both dataframes, which allows for a merge. The type of merge to apply depends on the desired content of the final dataframe. Since we want only the information present in both dataframes to be included, an 'inner' merge is the most suitable option.

In [22]:
# merging data sets using inner merge
df_ords_prods_merge = df_ords_prods_cleaned.merge(df_prods_cleaned, on = 'product_id', how = 'inner', indicator = True)
df_ords_prods_merge.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered,product_name,aisle_id,department_id,prices,_merge
0,2539329,1,1,2,8,NaN,True,196,1,0,Soda,77,7,9.0,both
1,2539329,1,1,2,8,NaN,True,14084,2,0,Organic Unsweetened Vanilla Almond Milk,91,16,12.5,both
2,2539329,1,1,2,8,NaN,True,12427,3,0,Original Beef Jerky,23,19,4.4,both
3,2539329,1,1,2,8,NaN,True,26088,4,0,Aged White Cheddar Popcorn,23,19,4.7,both
4,2539329,1,1,2,8,NaN,True,26405,5,0,XL Pick-A-Size Paper Towel Rolls,54,17,1.0,both


In [23]:
df_ords_prods_merge.shape

(32404859, 15)

##### There are few rows missing in the above merge when compared to df_ords_prods_cleaned. We will find the no of missing rows below:

In [25]:
# Finding the missing rows
missing_product_ids = set(df_ords_prods_cleaned['product_id']) - set(df_prods_cleaned['product_id'])
unmatched_records = df_ords_prods_cleaned[df_ords_prods_cleaned['product_id'].isin(missing_product_ids)]
unmatched_records.head()

,order_id,user_id,order_number,order_dow,order_hour_of_day,days_since_prior_order,first_order,product_id,add_to_cart_order,reordered
2963,7099,27,63,3,10,1.0,False,6799,1,0
3205,1837192,27,80,2,8,6.0,False,6799,9,1
3253,3331846,28,3,0,16,2.0,False,34,6,0
3395,1385910,28,21,2,17,1.0,False,34,1,1
4694,2873174,38,1,0,10,NaN,True,116,11,0


In [26]:
# Product_id missing in data set
print(unmatched_records['product_id'].unique())

[ 6799    34   116  1511  2240  4790 26519  3230  3159   262  1780   525
 38183  2586  4283 40440  3736    69]


In [27]:
# Total no of missing rows
print(len(unmatched_records))

30200


##### In the code provided, the intended merge operation is an inner merge. This type of merge retains only the rows that have matching product_id values in both dataframes. The results of the merge will show how many rows came from both data frames and the _merge column indicates the origin of each rows appeared in both dataframe (matched on product_id). If all rows are 'both;, that means every row had a matching entry in both dataframes. Looking at the no of rows in initial df_ords_prods_cleaned exported data there are 32434489 rows and after merge there are only 32404859 rows, some records are missing i.e. 30200 from the exported data. Since the missing data is less than 1% of data from df_ords_prods_cleaned and getting rid of these data should not likely affect our further analysis and so further investigating into missing data won't be required.

### 06 Check the results

In [30]:
df_ords_prods_merge['_merge'].value_counts()

_merge
both          32404859
left_only            0
right_only           0
Name: count, dtype: int64

### 07 Exporting data to pickle format

In [32]:
df_ords_prods_merge.to_pickle(os.path.join(path, '02 Data', 'Prepared Data', 'ords_prods_merge.pkl'))